###1. Load model và chạy thử lấy output

####TẢI DATA + GIẢI NÉN

In [ ]:
!wget http://cg.cs.tsinghua.edu.cn/download/DeepStab.zip

--2024-09-02 13:26:22--  http://cg.cs.tsinghua.edu.cn/download/DeepStab.zip
Resolving cg.cs.tsinghua.edu.cn (cg.cs.tsinghua.edu.cn)... 101.6.15.70, 2402:f000:1:402:101:6:15:70
Connecting to cg.cs.tsinghua.edu.cn (cg.cs.tsinghua.edu.cn)|101.6.15.70|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cg.cs.tsinghua.edu.cn/download/DeepStab.zip [following]
--2024-09-02 13:26:23--  https://cg.cs.tsinghua.edu.cn/download/DeepStab.zip
Connecting to cg.cs.tsinghua.edu.cn (cg.cs.tsinghua.edu.cn)|101.6.15.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8459488211 (7.9G) [application/zip]
Saving to: ‘DeepStab.zip’

DeepStab.zip        100%[===================>]   7.88G  8.17MB/s    in 17m 7s  

2024-09-02 13:43:32 (7.85 MB/s) - ‘DeepStab.zip’ saved [8459488211/8459488211]



In [ ]:
!unzip /content/DeepStab.zip

Archive:  /content/DeepStab.zip
   creating: DeepStab/
  inflating: DeepStab/Readme.txt     
   creating: DeepStab/stable/
  inflating: DeepStab/stable/1.avi   
  inflating: DeepStab/stable/10.avi  
  inflating: DeepStab/stable/11.avi  
  inflating: DeepStab/stable/12.avi  
  inflating: DeepStab/stable/13.avi  
  inflating: DeepStab/stable/14.avi  
  inflating: DeepStab/stable/15.avi  
  inflating: DeepStab/stable/16.avi  
  inflating: DeepStab/stable/17.avi  
  inflating: DeepStab/stable/18.avi  
  inflating: DeepStab/stable/19.avi  
  inflating: DeepStab/stable/2.avi   
  inflating: DeepStab/stable/20.avi  
  inflating: DeepStab/stable/21.avi  
  inflating: DeepStab/stable/22.avi  
  inflating: DeepStab/stable/23.avi  
  inflating: DeepStab/stable/24.avi  
  inflating: DeepStab/stable/25.avi  
  inflating: DeepStab/stable/26.avi  
  inflating: DeepStab/stable/27.avi  
  inflating: DeepStab/stable/28.avi  
  inflating: DeepStab/stable/29.avi  
  inflating: DeepStab/stable/3.avi   
  i

####MODEL

In [ ]:
import math
import numpy as np

import torch
import torch.nn as nn


class Encoder(nn.Module):
    def __init__(self, in_channels=3, depth=3):
        super(Encoder, self).__init__()

        # Shuffle pixels to expand in channel dimension
        # shuffler_list = [PixelShuffle(0.5) for i in range(depth)]
        # self.shuffler = nn.Sequential(*shuffler_list)
        self.shuffler = PixelShuffle(1 / 2**depth)

        relu = nn.LeakyReLU(0.2, True)

        # FF_RCAN or FF_Resblocks
        self.interpolate = Interpolation(5, 12, in_channels * (4**depth), act=relu)

    def forward(self, x1, x2):
        """
        Encoder: Shuffle-spread --> Feature Fusion --> Return fused features
        """
        feats1 = self.shuffler(x1)
        feats2 = self.shuffler(x2)

        feats = self.interpolate(feats1, feats2)

        return feats


class Decoder(nn.Module):
    def __init__(self, depth=3):
        super(Decoder, self).__init__()

        # shuffler_list = [PixelShuffle(2) for i in range(depth)]
        # self.shuffler = nn.Sequential(*shuffler_list)
        self.shuffler = PixelShuffle(2**depth)

    def forward(self, feats):
        out = self.shuffler(feats)
        return out


class CAIN(nn.Module):
    def __init__(self,training, depth=3):
        super(CAIN, self).__init__()

        self.encoder = Encoder(in_channels=3, depth=depth)
        self.decoder = Decoder(depth=depth)
        self.training = training
    def forward(self, x1, x2):
        x1, m1 = sub_mean(x1)
        x2, m2 = sub_mean(x2)

        if not self.training:
            paddingInput, paddingOutput = InOutPaddings(x1)
            x1 = paddingInput(x1)
            x2 = paddingInput(x2)

        feats = self.encoder(x1, x2)
        out = self.decoder(feats)

        if not self.training:
            out = paddingOutput(out)

        mi = (m1 + m2) / 2
        out += mi

        return out, feats


import math

import torch
import torch.nn as nn
import torch.nn.functional as F

def sub_mean(x):
    mean = x.mean(2, keepdim=True).mean(3, keepdim=True)
    x -= mean
    return x, mean

def InOutPaddings(x):
    w, h = x.size(3), x.size(2)
    padding_width, padding_height = 0, 0
    if w != ((w >> 7) << 7):
        padding_width = (((w >> 7) + 1) << 7) - w
    if h != ((h >> 7) << 7):
        padding_height = (((h >> 7) + 1) << 7) - h
    paddingInput = nn.ReflectionPad2d(padding=[padding_width // 2, padding_width - padding_width // 2,
                                               padding_height // 2, padding_height - padding_height // 2])
    paddingOutput = nn.ReflectionPad2d(padding=[0 - padding_width // 2, padding_width // 2 - padding_width,
                                                0 - padding_height // 2, padding_height // 2 - padding_height])
    return paddingInput, paddingOutput


class ConvNorm(nn.Module):
    def __init__(self, in_feat, out_feat, kernel_size, stride=1, norm=False):
        super(ConvNorm, self).__init__()

        reflection_padding = kernel_size // 2
        self.reflection_pad = nn.ReflectionPad2d(reflection_padding)
        self.conv = nn.Conv2d(in_feat, out_feat, stride=stride, kernel_size=kernel_size, bias=True)

        self.norm = norm
        if norm == 'IN':
            self.norm = nn.InstanceNorm2d(out_feat, track_running_stats=True)
        elif norm == 'BN':
            self.norm = nn.BatchNorm2d(out_feat)

    def forward(self, x):
        out = self.reflection_pad(x)
        out = self.conv(out)
        if self.norm:
            out = self.norm(out)
        return out


class UpConvNorm(nn.Module):
    def __init__(self, in_channels, out_channels, mode='transpose', norm=False):
        super(UpConvNorm, self).__init__()

        if mode == 'transpose':
            self.upconv = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=4, stride=2, padding=1)
        elif mode == 'shuffle':
            self.upconv = nn.Sequential(
                ConvNorm(in_channels, 4*out_channels, kernel_size=3, stride=1, norm=norm),
                PixelShuffle(2))
        else:
            # out_channels is always going to be the same as in_channels
            self.upconv = nn.Sequential(
                nn.Upsample(mode='bilinear', scale_factor=2, align_corners=False),
                ConvNorm(in_channels, out_channels, kernel_size=1, stride=1, norm=norm))

    def forward(self, x):
        out = self.upconv(x)
        return out



class meanShift(nn.Module):
    def __init__(self, rgbRange, rgbMean, sign, nChannel=3):
        super(meanShift, self).__init__()
        if nChannel == 1:
            l = rgbMean[0] * rgbRange * float(sign)

            self.shifter = nn.Conv2d(1, 1, kernel_size=1, stride=1, padding=0)
            self.shifter.weight.data = torch.eye(1).view(1, 1, 1, 1)
            self.shifter.bias.data = torch.Tensor([l])
        elif nChannel == 3:
            r = rgbMean[0] * rgbRange * float(sign)
            g = rgbMean[1] * rgbRange * float(sign)
            b = rgbMean[2] * rgbRange * float(sign)

            self.shifter = nn.Conv2d(3, 3, kernel_size=1, stride=1, padding=0)
            self.shifter.weight.data = torch.eye(3).view(3, 3, 1, 1)
            self.shifter.bias.data = torch.Tensor([r, g, b])
        else:
            r = rgbMean[0] * rgbRange * float(sign)
            g = rgbMean[1] * rgbRange * float(sign)
            b = rgbMean[2] * rgbRange * float(sign)
            self.shifter = nn.Conv2d(6, 6, kernel_size=1, stride=1, padding=0)
            self.shifter.weight.data = torch.eye(6).view(6, 6, 1, 1)
            self.shifter.bias.data = torch.Tensor([r, g, b, r, g, b])

        # Freeze the meanShift layer
        for params in self.shifter.parameters():
            params.requires_grad = False

    def forward(self, x):
        x = self.shifter(x)

        return x


""" CONV - (BN) - RELU - CONV - (BN) """
class ResBlock(nn.Module):
    def __init__(self, in_feat, out_feat, kernel_size=3, reduction=False, bias=True, # 'reduction' is just for placeholder
                 norm=False, act=nn.ReLU(True), downscale=False):
        super(ResBlock, self).__init__()

        self.body = nn.Sequential(
            ConvNorm(in_feat, out_feat, kernel_size=kernel_size, stride=2 if downscale else 1),
            act,
            ConvNorm(out_feat, out_feat, kernel_size=kernel_size, stride=1)
        )

        self.downscale = None
        if downscale:
            self.downscale = nn.Conv2d(in_feat, out_feat, kernel_size=1, stride=2)

    def forward(self, x):
        res = x
        out = self.body(x)
        if self.downscale is not None:
            res = self.downscale(res)
        out += res

        return out


## Channel Attention (CA) Layer
class CALayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(CALayer, self).__init__()
        # global average pooling: feature --> point
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        # feature channel downscale and upscale --> channel weight
        self.conv_du = nn.Sequential(
            nn.Conv2d(channel, channel // reduction, 1, padding=0, bias=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(channel // reduction, channel, 1, padding=0, bias=True),
            nn.Sigmoid()
        )

    def forward(self, x):
        y = self.avg_pool(x)
        y = self.conv_du(y)
        return x * y, y


## Residual Channel Attention Block (RCAB)
class RCAB(nn.Module):
    def __init__(self, in_feat, out_feat, kernel_size, reduction, bias=True,
            norm=False, act=nn.ReLU(True), downscale=False, return_ca=False):
        super(RCAB, self).__init__()

        self.body = nn.Sequential(
            ConvNorm(in_feat, out_feat, kernel_size, stride=2 if downscale else 1, norm=norm),
            act,
            ConvNorm(out_feat, out_feat, kernel_size, stride=1, norm=norm),
            CALayer(out_feat, reduction)
        )
        self.downscale = downscale
        if downscale:
            self.downConv = nn.Conv2d(in_feat, out_feat, kernel_size=3, stride=2, padding=1)
        self.return_ca = return_ca

    def forward(self, x):
        res = x
        out, ca = self.body(x)
        if self.downscale:
            res = self.downConv(res)
        out += res

        if self.return_ca:
            return out, ca
        else:
            return out


## Residual Group (RG)
class ResidualGroup(nn.Module):
    def __init__(self, Block, n_resblocks, n_feat, kernel_size, reduction, act, norm=False):
        super(ResidualGroup, self).__init__()

        modules_body = [Block(n_feat, n_feat, kernel_size, reduction, bias=True, norm=norm, act=act)
            for _ in range(n_resblocks)]
        modules_body.append(ConvNorm(n_feat, n_feat, kernel_size, stride=1, norm=norm))
        self.body = nn.Sequential(*modules_body)

    def forward(self, x):
        res = self.body(x)
        res += x
        return res


def pixel_shuffle(input, scale_factor):
    batch_size, channels, in_height, in_width = input.size()

    out_channels = int(int(channels / scale_factor) / scale_factor)
    out_height = int(in_height * scale_factor)
    out_width = int(in_width * scale_factor)

    if scale_factor >= 1:
        input_view = input.contiguous().view(batch_size, out_channels, scale_factor, scale_factor, in_height, in_width)
        shuffle_out = input_view.permute(0, 1, 4, 2, 5, 3).contiguous()
    else:
        block_size = int(1 / scale_factor)
        input_view = input.contiguous().view(batch_size, channels, out_height, block_size, out_width, block_size)
        shuffle_out = input_view.permute(0, 1, 3, 5, 2, 4).contiguous()

    return shuffle_out.view(batch_size, out_channels, out_height, out_width)


class PixelShuffle(nn.Module):
    def __init__(self, scale_factor):
        super(PixelShuffle, self).__init__()
        self.scale_factor = scale_factor

    def forward(self, x):
        return pixel_shuffle(x, self.scale_factor)
    def extra_repr(self):
        return 'scale_factor={}'.format(self.scale_factor)


def conv(in_channels, out_channels, kernel_size,
         stride=1, bias=True, groups=1):
    return nn.Conv2d(
        in_channels,
        out_channels,
        kernel_size=kernel_size,
        padding=kernel_size//2,
        stride=1,
        bias=bias,
        groups=groups)


def conv1x1(in_channels, out_channels, stride=1, bias=True, groups=1):
    return nn.Conv2d(
        in_channels,
        out_channels,
        kernel_size=1,
        stride=stride,
        bias=bias,
        groups=groups)

def conv3x3(in_channels, out_channels, stride=1,
            padding=1, bias=True, groups=1):
    return nn.Conv2d(
        in_channels,
        out_channels,
        kernel_size=3,
        stride=stride,
        padding=padding,
        bias=bias,
        groups=groups)

def conv5x5(in_channels, out_channels, stride=1,
            padding=2, bias=True, groups=1):
    return nn.Conv2d(
        in_channels,
        out_channels,
        kernel_size=5,
        stride=stride,
        padding=padding,
        bias=bias,
        groups=groups)

def conv7x7(in_channels, out_channels, stride=1,
            padding=3, bias=True, groups=1):
    return nn.Conv2d(
        in_channels,
        out_channels,
        kernel_size=7,
        stride=stride,
        padding=padding,
        bias=bias,
        groups=groups)

def upconv2x2(in_channels, out_channels, mode='shuffle'):
    if mode == 'transpose':
        return nn.ConvTranspose2d(
            in_channels,
            out_channels,
            kernel_size=4,
            stride=2,
            padding=1)
    elif mode == 'shuffle':
        return nn.Sequential(
            conv3x3(in_channels, 4*out_channels),
            PixelShuffle(2))
    else:
        # out_channels is always going to be the same as in_channels
        return nn.Sequential(
            nn.Upsample(mode='bilinear', scale_factor=2, align_corners=False),
            conv1x1(in_channels, out_channels))



class Interpolation(nn.Module):
    def __init__(self, n_resgroups, n_resblocks, n_feats,
                 reduction=16, act=nn.LeakyReLU(0.2, True), norm=False):
        super(Interpolation, self).__init__()

        # define modules: head, body, tail
        self.headConv = conv3x3(n_feats * 2, n_feats)

        modules_body = [
            ResidualGroup(
                RCAB,
                n_resblocks=n_resblocks,
                n_feat=n_feats,
                kernel_size=3,
                reduction=reduction,
                act=act,
                norm=norm)
            for _ in range(n_resgroups)]
        self.body = nn.Sequential(*modules_body)

        self.tailConv = conv3x3(n_feats, n_feats)

    def forward(self, x0, x1):
        # Build input tensor
        x = torch.cat([x0, x1], dim=1)
        x = self.headConv(x)

        res = self.body(x)
        res += x

        out = self.tailConv(res)
        return out


class Interpolation_res(nn.Module):
    def __init__(self, n_resgroups, n_resblocks, n_feats,
                 act=nn.LeakyReLU(0.2, True), norm=False):
        super(Interpolation_res, self).__init__()

        # define modules: head, body, tail (reduces concatenated inputs to n_feat)
        self.headConv = conv3x3(n_feats * 2, n_feats)

        modules_body = [ResidualGroup(ResBlock, n_resblocks=n_resblocks, n_feat=n_feats, kernel_size=3,
                            reduction=0, act=act, norm=norm)
                        for _ in range(n_resgroups)]
        self.body = nn.Sequential(*modules_body)

        self.tailConv = conv3x3(n_feats, n_feats)

    def forward(self, x0, x1):
        # Build input tensor
        x = torch.cat([x0, x1], dim=1)
        x = self.headConv(x)

        res = x
        for m in self.body:
            res = m(res)
        res += x

        x = self.tailConv(res)

        return x

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init


class ChannelAttention(nn.Module):
    def __init__(self, in_channels, reduction_ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction_ratio),
            nn.ReLU(),
            nn.Linear(in_channels // reduction_ratio, in_channels),
            nn.Sigmoid()
        )

    def forward(self, x):
        avg_pool = self.avg_pool(x).view(x.size(0), -1)
        channel_att = self.fc(avg_pool).view(x.size(0), x.size(1), 1, 1)
        return x * channel_att

class ResidualBlockWithChannelAttention(nn.Module):
    def __init__(self, in_features=32, out_features=32):
        super(ResidualBlockWithChannelAttention, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_features, out_features, kernel_size=1, stride=1, padding=0),
            nn.LeakyReLU(0.2)
        )
        self.channel_attention = ChannelAttention(out_features)

    def forward(self, x):
        x1 = self.conv(x)
        x1 = self.channel_attention(x1)
        return x1 + x

class ConvBlockWithChannelAttention(nn.Module):
    def __init__(self, in_features, out_features):
        super(ConvBlockWithChannelAttention, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(in_features, out_features, kernel_size=1, stride=1, padding=0),
            nn.LeakyReLU(0.2)
        )

        self.channel_attention = ChannelAttention(out_features)

    def forward(self, x):
        x1 = self.conv(x)
        x1 = self.channel_attention(x1)
        return x1

class ResNet(nn.Module):
    def __init__(self, hidden_channels):
        super(ResNet, self).__init__()
        self.conv_block = ConvBlockWithChannelAttention(15, hidden_channels)
        self.res_block1 = ResidualBlockWithChannelAttention(hidden_channels, hidden_channels)
        self.res_block2 = ResidualBlockWithChannelAttention(hidden_channels, hidden_channels)
        self.res_block3 = ResidualBlockWithChannelAttention(hidden_channels, hidden_channels)
        self.res_block4 = ResidualBlockWithChannelAttention(hidden_channels, hidden_channels)
        self.res_block5 = ResidualBlockWithChannelAttention(hidden_channels, hidden_channels)
        self.conv_final = nn.Conv2d(hidden_channels, 3, kernel_size=1, stride=1, padding=0)

        # Initialize the weights with Kaiming initialization
        self.apply(self._init_weights)

        total_trainable_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        print(f"Total ResNet Parameters: {total_trainable_params}")

    def _init_weights(self, m):
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d)):
            init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='leaky_relu')

    def forward(self, x):
        # ConvBlock
        x = self.conv_block(x)

        # Residual Block 1
        x = self.res_block1(x)

        # Residual Block 2
        x = self.res_block2(x)

        # Residual Block 3
        x = self.res_block3(x)

        # Residual Block 4
        x = self.res_block4(x)

        # Residual Block 5
        x = self.res_block5(x)

        # Final Convolutional Layer
        x = self.conv_final(x)

        return x

class Get_gradient(nn.Module):
    def __init__(self):
        super(Get_gradient, self).__init__()
        kernel_v = [[0, -1, 0],
                    [0, 0, 0],
                    [0, 1, 0]]
        kernel_h = [[0, 0, 0],
                    [-1, 0, 1],
                    [0, 0, 0]]
        kernel_h = torch.FloatTensor(kernel_h).unsqueeze(0).unsqueeze(0)
        kernel_v = torch.FloatTensor(kernel_v).unsqueeze(0).unsqueeze(0)
        self.weight_h = nn.Parameter(data = kernel_h, requires_grad = False).cuda()
        self.weight_v = nn.Parameter(data = kernel_v, requires_grad = False).cuda()

    def forward(self, x):
        x0 = x[:, 0]
        x1 = x[:, 1]
        x2 = x[:, 2]
        x0_v = F.conv2d(x0.unsqueeze(1), self.weight_v, padding=2)
        x0_h = F.conv2d(x0.unsqueeze(1), self.weight_h, padding=2)

        x1_v = F.conv2d(x1.unsqueeze(1), self.weight_v, padding=2)
        x1_h = F.conv2d(x1.unsqueeze(1), self.weight_h, padding=2)

        x2_v = F.conv2d(x2.unsqueeze(1), self.weight_v, padding=2)
        x2_h = F.conv2d(x2.unsqueeze(1), self.weight_h, padding=2)

        x0 = torch.sqrt(torch.pow(x0_v, 2) + torch.pow(x0_h, 2) + 1e-6)
        x1 = torch.sqrt(torch.pow(x1_v, 2) + torch.pow(x1_h, 2) + 1e-6)
        x2 = torch.sqrt(torch.pow(x2_v, 2) + torch.pow(x2_h, 2) + 1e-6)

        x = torch.cat([x0, x1, x2], dim=1)
        return x

In [ ]:
import numpy as np
import cv2
import argparse
import os
import torch
import cv2
H,W = 256,256
device = 'cuda'


def parse_args():
    parser = argparse.ArgumentParser(description='Video Stabilization using CAIN')
    parser.add_argument('--in_path', type=str, help='Input video file path')
    parser.add_argument('--out_path', type=str, help='Output stabilized video file path')
    return parser.parse_args()

def save_video(frames, path):
    frame_count,h,w,_ = frames.shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(path, fourcc, 30.0, (w,h))
    for idx in range(frame_count):
        out.write(frames[idx,...])
    out.release()

def stabilize(in_path,out_path):

    if not os.path.exists(in_path):
        print(f"The input file '{in_path}' does not exist.")
        exit()
    _,ext = os.path.splitext(in_path)
    if ext not in ['.mp4','.avi']:
        print(f"The input file '{in_path}' is not a supported video file (only .mp4 and .avi are supported).")
        exit()

    #Load frames and stardardize
    cap = cv2.VideoCapture(in_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = np.zeros((frame_count,H,W,3),np.float32)
    for i in range(frame_count):
        ret,img = cap.read()
        if not ret:
            break
        img = cv2.resize(img,(W,H))
        img = ((img / 255.0) * 2) - 1
        frames[i,...] = img
        print(frames.size)
    # stabilize video
    SKIP = 1
    ITER = 3
    interpolated = frames.copy()
    for iter in range(ITER):
        print(iter)
        temp = interpolated.copy()
        for frame_idx in range(2,frame_count - 2):
            torch.cuda.empty_cache()
            ft_minus = torch.from_numpy(interpolated[frame_idx - 1,...]).permute(2,0,1).unsqueeze(0).to(device)
            ft = torch.from_numpy(frames[frame_idx]).permute(2,0,1).unsqueeze(0).to(device)
            ft_plus = torch.from_numpy(interpolated[frame_idx + 1,...]).permute(2,0,1).unsqueeze(0).to(device)
            with torch.no_grad():
                fout,features = cain(ft_minus,ft_plus)
                #refinement step
                if iter == 2:
                    ft_2 = torch.from_numpy(frames[frame_idx -2,...]).permute(2,0,1).unsqueeze(0).to(device)
                    ft_1 = torch.from_numpy(frames[frame_idx -1,...]).permute(2,0,1).unsqueeze(0).to(device)
                    ftplus1 = torch.from_numpy(frames[frame_idx +1,...]).permute(2,0,1).unsqueeze(0).to(device)
                    ftplus2 = torch.from_numpy(frames[frame_idx +2,...]).permute(2,0,1).unsqueeze(0).to(device)
                    resnet_input = torch.cat([ft_2, ft_1, fout, ftplus1, ftplus2],dim = 1)
                    fout = resnet(resnet_input)
            temp[frame_idx,...] = fout.cpu().squeeze(0).permute(1,2,0).numpy()
            img  = (((fout.cpu().squeeze(0).permute(1,2,0).numpy() + 1) / 2)*255.0)
            img  = np.clip(img,0,255).astype(np.uint8)
        interpolated = temp.copy()
    stable_frames = np.clip((255 *(interpolated + 1) / 2),0,255).astype(np.uint8)
    save_video(stable_frames,out_path)

In [ ]:
# tải trọng số

!gdown --id 1GJY31u3JMGansi0glQW1YHLKpPFQqlnO


/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1GJY31u3JMGansi0glQW1YHLKpPFQqlnO
From (redirected): https://drive.google.com/uc?id=1GJY31u3JMGansi0glQW1YHLKpPFQqlnO&confirm=t&uuid=8335219c-c0bf-40a4-8512-08036d64c1c9
To: /content/ckpts.rar
100% 160M/160M [00:02<00:00, 71.4MB/s]


In [ ]:
#giải nén rar file

!unrar x /content/ckpts.rar



UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/ckpts.rar

Creating    ckpts                                                     OK
Creating    ckpts/CAIN                                                OK
Extracting  ckpts/CAIN/pretrained_cain.pth                                 2%  5%  7% 10% 13% 15% 18% 20% 23% 26% 28% 31% 34% 36% 39% 41% 44% 47% 49% 52% 54% 57% 60% 62% 65% 68% 70% 73% 75% 78% 81% 83% 86% 88% 91% 94% 96% 99%  OK 
Creating    ckpts/ResNet                                              OK
Extracting  ckpts/ResNet/resnet_5.pth                                     99%  OK 
All OK


In [ ]:
resnet = ResNet(hidden_channels=64)
resnet = torch.nn.DataParallel(resnet).to(device).eval()
cain = CAIN(training= False,depth=3)
cain = torch.nn.DataParallel(cain).to(device).eval()
cain_ckpt = torch.load('/content/ckpts/CAIN/pretrained_cain.pth')
cain.load_state_dict(cain_ckpt['state_dict'])
print('Loaded pretrained CAIN')
# Load ResNet checkpoints
state_dict = torch.load('/content/ckpts/ResNet/resnet_5.pth')
resnet.load_state_dict(state_dict['model'])

Total ResNet Parameters: 25499


<ipython-input-7-3ab71ed9747d>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cain_ckpt = torch.load('/content/ckpts/CAIN/pretrained_cain.pth')


Loaded pretrained CAIN


<ipython-input-7-3ab71ed9747d>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('/content/ckpts/ResNet/resnet_5.pth')


<All keys matched successfully>

In [ ]:
input = "/content/DeepStab/unstable/1.avi"
ouput = "/content/stable_result.avi"

In [ ]:
stabilize(input, ouput)

91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
91422720
9

tải file output về máy để xem do không thể xem trên cloud

###2. Thử nghiệm train + test model

###CONVERT VIDEO DATA TO IMAGE DATA

In [ ]:
import cv2
import os

def video_to_images(video_path, output_dir):
    """Converts a video to individual frames and saves them in the output directory.

    Args:
        video_path (str): Path to the video file.
        output_dir (str): Path to the directory where frames will be saved.
    """
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    output_video_dir = os.path.join(output_dir, video_name)
    os.makedirs(output_video_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_filename = os.path.join(output_video_dir, f"frame{frame_count:04d}.jpg")
        cv2.imwrite(frame_filename, frame)
        frame_count += 1
    cap.release()

dataset_path = "/content/DeepStab/unstable"  # Replace with your dataset path
output_dir = "/content/DeepStab/unstable_frames"  # Replace with your desired output directory

for video_file in os.listdir(dataset_path):
    if video_file.endswith(('.mp4', '.avi')):  # Adjust video extensions as needed
        video_path = os.path.join(dataset_path, video_file)
        video_to_images(video_path, output_dir)


In [ ]:
dataset_path = "/content/DeepStab/stable"  # Replace with your dataset path
output_dir = "/content/DeepStab/stable_frames"  # Replace with your desired output directory

for video_file in os.listdir(dataset_path):
    if video_file.endswith(('.mp4', '.avi')):  # Adjust video extensions as needed
        video_path = os.path.join(dataset_path, video_file)
        video_to_images(video_path, output_dir)

###MAKE DATASET

In [ ]:
### python lib
import os, sys, math, random, glob, cv2, argparse, natsort
import numpy as np

### torch lib
import torch
import torch.utils.data as data
import torchvision.transforms as tv
from torch.utils.data.sampler import Sampler
from torch.utils.data import DataLoader

def read_img(filename, resize= False, scale_factor= 1.0):

    ## read image and convert to RGB in [0, 1]

    img = cv2.imread(filename)
    if resize:
        img = cv2.resize(img, (0,0), fx= scale_factor, fy= scale_factor)
        #print(img.shape)

    if img is None:
        raise Exception("Image %s does not exist" %filename)

    #img = img[:, :, ::-1] ## BGR to RGB

    img = np.float32(img) / 255.0

    return img

crop_thingies = (0, 0, 0, 0)

class RandomCrop(object):
    def __init__(self, image_size, crop_size):
        self.ch, self.cw = crop_size
        ih, iw = image_size

        self.h1 = random.randint(0, ih - self.ch - 0)
        self.w1 = random.randint(0, iw - self.cw - 0)

        self.h2 = self.h1 + self.ch
        self.w2 = self.w1 + self.cw
        global crop_thingies
        crop_thingies = (self.h1, self.h2, self.w1, self.w2)

    def __call__(self, img):
        #print("Came in to cropper...", crop_thingies)
        #sys.exit()
        if len(img.shape) == 3:
            return img[self.h1 : self.h2, self.w1 : self.w2, :]
        else:
            return img[self.h1 : self.h2, self.w1 : self.w2]


class FixedCrop(object):
    def __init__(self, crop_size, mh, mw):
        self.ch, self.cw = crop_size
        self.h1 = mh
        self.w1 = mw
        self.h2 = self.h1 + self.ch
        self.w2 = self.w1 + self.cw
        #global crop_thingies
        #crop_thingies = (self.h1, self.h2, self.w1, self.w2)

    def __call__(self, img):
        #print("Came in to cropper...", crop_thingies)
        #sys.exit()
        if len(img.shape) == 3:
            return img[self.h1 : self.h2, self.w1 : self.w2, :]
        else:
            return img[self.h1 : self.h2, self.w1 : self.w2]



class MultiFramesDataset(data.Dataset):

    def __init__(self, mode= 'train', opts= None, get_lbls= False, prev_iter_counter= 0):
        super(MultiFramesDataset, self).__init__()
        self.mode = mode
        self.ip_dir = opts.data_dir
        self.dataset_task_list = []
        self.num_frames = []
        self.get_lbls = get_lbls
        self.iter_counter = prev_iter_counter
        if self.mode == "train":
            self.sample_frames = opts.sample_frames
            self.geometry_aug = opts.geometry_aug
            self.scale_min = opts.scale_min
            self.scale_max = opts.scale_max
            self.crop_size = opts.crop_size
            self.order_aug = opts.order_aug
            self.size_multiplier = opts.size_multiplier

            self.dataset_task_list = natsort.natsorted(glob.glob(os.path.join(self.ip_dir + "*/")))
            self.dataset_lbls_list = natsort.natsorted(glob.glob(os.path.join(opts.train_lbl_dir + "*/")))
            self.shuffled_inds = np.arange(len(self.dataset_task_list))
            np.random.shuffle(self.shuffled_inds)
            self.num_frames = [len(os.listdir(dir)) for dir in self.dataset_task_list]
            #list_lbl = natsort.natsorted(glob.glob(os.path.join(self.lbl_dir + "*/")))
        elif self.mode == "val":
            self.dataset_task_list = natsort.natsorted(glob.glob(os.path.join(self.ip_dir + "*/")))
            #self.dataset_task_list = [os.path.join(self.ip_dir+i+'/') for i in ['Crowd_7','Parallax_10','Regular_16','Running_16','Zooming_11']]
            self.num_frames = [len(os.listdir(dir)) for dir in self.dataset_task_list]
            # print(self.dataset_task_list)
        else:
            raise ValueError("Only train and val mode is implemented at the moment...")

        print("+"*20, "Total task_list:", "+"*20)
        video_names = []
        for task in self.dataset_task_list:
            video_names.append(task.split("/")[-2])
        print(video_names)
        print("+"*50)
        self.num_tasks = len(self.dataset_task_list)


        global crop_thingies

        print("[%s] Total %d videos (%d frames)" %(self.__class__.__name__, len(self.dataset_task_list), sum(self.num_frames)))


    def __len__(self):
        return len(self.dataset_task_list)


    def __getitem__(self, index):
        meta_data = {}
        meta_data['idx'] = index
        ## random select starting frame index t between [0, N - number_of_sample_frames] for "mode = "train" | "validate""
        if self.mode == "train":
            index_ = self.shuffled_inds[index]
            N = self.num_frames[index_]
            T = random.randint(0, N - self.sample_frames)
            #T = 0 #### Change this again for later experiments to randomize the frames for training
            meta_data['starting_frame'] = T

            input_dir = self.dataset_task_list[index_]
            if self.get_lbls:
                lbl_dir = self.dataset_lbls_list[index_]
            meta_data['unstable_video_path'] = input_dir
            if self.get_lbls:
                meta_data['stable_video_path'] = lbl_dir

            ## sample from T to T + #sample_frames - 1
            frame_ip = []
            if self.get_lbls:
                frame_lbl = []
            ip_frame_list = natsort.natsorted(glob.glob(os.path.join(input_dir, "*.*")))
            if self.get_lbls:
                lbl_frame_list = natsort.natsorted(glob.glob(os.path.join(lbl_dir, "*.*")))
            for t in range(T, T + self.sample_frames):
                if self.iter_counter < 200000:
                    frame_ip.append(read_img(ip_frame_list[t], True, 1.0))
                    if self.get_lbls:
                        frame_lbl.append(read_img(lbl_frame_list[t], True, 1.0))
                else:
                    frame_ip.append(read_img(ip_frame_list[t], False, 1.0))
                    if self.get_lbls:
                        frame_lbl.append(read_img(lbl_frame_list[t], False, 1.0))

            meta_data['ip_frame_paths'] = ip_frame_list[T:T + self.sample_frames]
            if self.get_lbls:
                meta_data['op_frame_paths'] = lbl_frame_list[T:T + self.sample_frames]

            self.iter_counter += 1

            ## data augmentation
            if self.geometry_aug:

                ## random scale
                H_in = frame_ip[0].shape[0]
                W_in = frame_ip[0].shape[1]

                sc = np.random.uniform(self.scale_min, self.scale_max)
                H_out = int(math.floor(H_in * sc))
                W_out = int(math.floor(W_in * sc))

                ## scaled size should be equal to opts.crop_size
                if H_out < W_out:
                    if H_out < self.crop_size:
                        H_out = self.crop_size
                        W_out = int(math.floor(W_in * float(H_out) / float(H_in)))
                else: ## W_out < H_out
                    if W_out < self.crop_size:
                        W_out = self.crop_size
                        H_out = int(math.floor(H_in * float(W_out) / float(W_in)))

                for t in range(self.sample_frames):
                    frame_ip[t] = cv2.resize(frame_ip[t], (W_out, H_out))
                    if self.get_lbls:
                        frame_lbl[t] = cv2.resize(frame_lbl[t], (W_out, H_out))
                meta_data['scale_factor'] = sc

            ## random crop
            cropper = RandomCrop(frame_ip[0].shape[:2], (self.crop_size, self.crop_size))


            for t in range(self.sample_frames):
                frame_ip[t] = cropper(frame_ip[t])
                if self.get_lbls:
                    frame_lbl[t] = cropper(frame_lbl[t])
                if frame_ip[t].shape[1] != self.crop_size and frame_ip[t].shape[0] != self.crop_size:
                    print("[MultiFrameDataset]: size mismatch occured... =>", frame_ip[t].shape)
                if self.get_lbls:
                        if frame_lbl[t].shape[1] != self.crop_size and frame_lbl[t].shape[0] != self.crop_size:
                            print("[MultiFrameDataset]: size mismatch occured... =>", frame_lbl[t].shape)
            meta_data['crop_coords'] = crop_thingies


            if self.geometry_aug:
                #meta_data['rotation'] = False
                ### random rotate
                rotate = random.randint(0, 3)
                if rotate != 0:
                    for t in range(self.sample_frames):
                        frame_ip[t] = np.rot90(frame_ip[t], rotate)
                        if self.get_lbls:
                            frame_lbl[t] = np.rot90(frame_lbl[t], rotate)
                    #meta_data['rotation'] = True

                ## horizontal flip
                if np.random.random() >= 0.5:
                    for t in range(self.sample_frames):
                        frame_ip[t] = cv2.flip(frame_ip[t], flipCode=0)
                        if self.get_lbls:
                            frame_lbl[t] = cv2.flip(frame_lbl[t], flipCode=0)
                    #meta_data['hflip'] = True


            if self.order_aug:
                ## reverse temporal order
                #meta_data['order'] = "normal"
                if np.random.random() >= 0.5:
                    #meta_data['order'] = "reversed"
                    frame_ip.reverse()
                    if self.get_lbls:
                        frame_lbl.reverse()

        elif self.mode == "val":
            input_dir = self.dataset_task_list[index]
            #lbl_dir = self.dataset_task_list[index][1]
            meta_data['unstable_video_path'] = input_dir
            meta_data['video_name'] = input_dir
            #meta_data['stable_video_path'] = lbl_dir

            ## sample from T to T + #sample_frames - 1
            frame_ip = []
            #frame_lbl = []
            ip_frame_list = natsort.natsorted(glob.glob(os.path.join(input_dir, "*.*")))
            #lbl_frame_list = natsort.natsorted(glob.glob(os.path.join(lbl_dir, "*.*")))
            for t in range(0, self.num_frames[index]):
                frame_ip.append(read_img(ip_frame_list[t]))
                #frame_lbl.append(read_img(lbl_frame_list[t]))
            meta_data['ip_frame_paths'] = ip_frame_list

        ### convert (H, W, C) array to (C, H, W) tensor
        X = []
        if self.mode == "train":
            if self.get_lbls:
                Y = []
            for t in range(len(frame_ip)):
                X.append(torch.from_numpy(frame_ip[t].transpose(2, 0, 1).astype(np.float32)))
                if self.get_lbls:
                    Y.append(torch.from_numpy(frame_lbl[t].transpose(2, 0, 1).astype(np.float32)))

            if self.get_lbls:
                return {'X': X, 'Y': Y, 'meta_data': meta_data}
            else:
                return {'X': X, 'meta_data': meta_data}

        else:
            for t in range(len(frame_ip)):
                X.append(torch.unsqueeze(torch.from_numpy(frame_ip[t].transpose(2, 0, 1).astype(np.float32)), 0))

            return {'X': X, 'meta_data': meta_data}


class MultiFramesDatasetHybrid(data.Dataset):

    def __init__(self, mode= 'train', opts= None, get_lbls= True):
        super(MultiFramesDatasetHybrid, self).__init__()
        self.ip_dir = opts.data_dir
        self.ip_dir_synth = opts.data_dir_synth
        self.lbl_dir = opts.train_lbl_dir
        self.sample_frames = opts.sample_frames
        self.geometry_aug = opts.geometry_aug
        self.scale_min = opts.scale_min
        self.scale_max = opts.scale_max
        self.crop_size = opts.crop_size
        self.order_aug = opts.order_aug
        self.size_multiplier = opts.size_multiplier
        self.mode = mode
        self.dataset_task_list = []
        self.num_frames = []
        self.get_lbls = get_lbls
        self.iter_counter = 0
        if self.mode == "train":
            self.dataset_task_list = natsort.natsorted(glob.glob(os.path.join(self.ip_dir + "*/")))
            self.dataset_syn_ip_list = natsort.natsorted(glob.glob(os.path.join(self.ip_dir_synth + "*/")))
            self.dataset_syn_lbls_list = natsort.natsorted(glob.glob(os.path.join(self.lbl_dir + "*/")))
            self.shuffled_inds = np.arange(len(self.dataset_task_list))
            np.random.shuffle(self.shuffled_inds)
            #list_lbl = natsort.natsorted(glob.glob(os.path.join(self.lbl_dir + "*/")))
        elif self.mode == "val":
            self.dataset_task_list = natsort.natsorted(glob.glob(os.path.join(self.ip_dir + "*/")))
        else:
            raise ValueError("Only train and val mode is implemented at the moment...")

        print("+"*20, "Total task_list:", "+"*20)
        video_names = []
        for task in self.dataset_task_list:
            video_names.append(task.split("/")[-2])
        print(video_names)
        print("+"*50)
        self.num_tasks = len(self.dataset_task_list)

        for ip in self.dataset_task_list:
            self.num_frames.append(len(natsort.natsorted(os.listdir(ip))))

        for ip in self.synth_list:
            self.num_frames_synth.append(len(natsort.natsorted(os.listdir(ip))))

        global crop_thingies

        print("[%s] Total %d videos (%d frames)" %(self.__class__.__name__, len(self.dataset_task_list), sum(self.num_frames)))


    def __len__(self):
        return len(self.dataset_task_list)


    def __getitem__(self, index):
        meta_data = {}
        meta_data['idx'] = index
        ## random select starting frame index t between [0, N - number_of_sample_frames] for "mode = "train" | "validate""
        if self.mode == "train":
            index_ = self.shuffled_inds[index]
            rand_num = random.randint(0, len(self.synth_list))
            N = self.num_frames[index_]
            Ns = self.num_frames_synth[rand_num]
            T = random.randint(0, N - self.sample_frames)
            Ts = random.randint(0, Ns - self.sample_frames)
            #T = 0 #### Change this again for later experiments to randomize the frames for training
            meta_data['starting_frame'] = T

            input_dir = self.dataset_task_list[index_]
            if self.get_lbls:
                syn_ip_dir = self.dataset_syn_ip_list[rand_num]
                lbl_dir = self.dataset_syn_lbls_list[rand_num]
            meta_data['unstable_video_path'] = input_dir
            if self.get_lbls:
                meta_data['stable_video_path'] = lbl_dir

            ## sample from T to T + #sample_frames - 1
            frame_ip = []
            if self.get_lbls:
                frame_syn = []
                frame_lbl = []
            ip_frame_list = natsort.natsorted(glob.glob(os.path.join(input_dir, "*.*")))
            if self.get_lbls:
                syn_ip_list = natsort.natsorted(glob.glob(os.path.join(syn_ip_dir, "*.*")))
                lbl_frame_list = natsort.natsorted(glob.glob(os.path.join(lbl_dir, "*.*")))

            for t in range(T, T + self.sample_frames):
                if self.iter_counter < 1000:
                    frame_ip.append(read_img(ip_frame_list[t], True, 0.3))
                    if self.get_lbls:
                        frame_syn.append(read_img(syn_ip_list[t], True, 0.3))
                        frame_lbl.append(read_img(lbl_frame_list[t], True, 0.3))
                elif self.iter_counter < 2000:
                    frame_ip.append(read_img(ip_frame_list[t], True, 0.4))
                    if self.get_lbls:
                        frame_syn.append(read_img(syn_ip_list[t], True, 0.4))
                        frame_lbl.append(read_img(lbl_frame_list[t], True, 0.4))
                elif self.iter_counter < 3000:
                    frame_ip.append(read_img(ip_frame_list[t], True, 0.45))
                    if self.get_lbls:
                        frame_syn.append(read_img(syn_ip_list[t], True, 0.45))
                        frame_lbl.append(read_img(lbl_frame_list[t], True, 0.45))
                elif self.iter_counter < 4000:
                    frame_ip.append(read_img(ip_frame_list[t], True, 0.5))
                    if self.get_lbls:
                        frame_syn.append(read_img(syn_ip_list[t], True, 0.5))
                        frame_lbl.append(read_img(lbl_frame_list[t], True, 0.5))
                elif self.iter_counter < 5000:
                    frame_ip.append(read_img(ip_frame_list[t], True, 0.55))
                    if self.get_lbls:
                        frame_syn.append(read_img(syn_ip_list[t], True, 0.55))
                        frame_lbl.append(read_img(lbl_frame_list[t], True, 0.55))
                elif self.iter_counter < 6000:
                    frame_ip.append(read_img(ip_frame_list[t], True, 0.6))
                    if self.get_lbls:
                        frame_syn.append(read_img(syn_ip_list[t], True, 0.6))
                        frame_lbl.append(read_img(lbl_frame_list[t], True, 0.6))
                elif self.iter_counter < 7000:
                    frame_ip.append(read_img(ip_frame_list[t], True, 0.7))
                    if self.get_lbls:
                        frame_syn.append(read_img(syn_ip_list[t], True, 0.7))
                        frame_lbl.append(read_img(lbl_frame_list[t], True, 0.7))
                elif self.iter_counter < 8000:
                    frame_ip.append(read_img(ip_frame_list[t], True, 0.8))
                    if self.get_lbls:
                        frame_syn.append(read_img(syn_ip_list[t], True, 0.8))
                        frame_lbl.append(read_img(lbl_frame_list[t], True, 0.8))
                elif self.iter_counter < 9000:
                    frame_ip.append(read_img(ip_frame_list[t], True, 0.9))
                    if self.get_lbls:
                        frame_syn.append(read_img(syn_ip_list[t], True, 0.9))
                        frame_lbl.append(read_img(lbl_frame_list[t], True, 0.9))
                else:
                    frame_ip.append(read_img(ip_frame_list[t], False, 1.0))
                    if self.get_lbls:
                        frame_syn.append(read_img(syn_ip_list[t], True, 1.0))
                        frame_lbl.append(read_img(lbl_frame_list[t], False, 1.0))

            meta_data['ip_frame_paths'] = ip_frame_list[T:T + self.sample_frames]
            if self.get_lbls:
                meta_data['op_frame_paths'] = lbl_frame_list[T:T + self.sample_frames]

            self.iter_counter += 1

            ## data augmentation
            if self.geometry_aug:

                ## random scale
                H_in = frame_ip[0].shape[0]
                W_in = frame_ip[0].shape[1]

                sc = np.random.uniform(self.scale_min, self.scale_max)
                H_out = int(math.floor(H_in * sc))
                W_out = int(math.floor(W_in * sc))

                ## scaled size should be equal to opts.crop_size
                if H_out < W_out:
                    if H_out < self.crop_size:
                        H_out = self.crop_size
                        W_out = int(math.floor(W_in * float(H_out) / float(H_in)))
                else: ## W_out < H_out
                    if W_out < self.crop_size:
                        W_out = self.crop_size
                        H_out = int(math.floor(H_in * float(W_out) / float(W_in)))

                for t in range(self.sample_frames):
                    frame_ip[t] = cv2.resize(frame_ip[t], (W_out, H_out))
                    if self.get_lbls:
                        frame_syn[t] = cv2.resize(frame_syn[t], (W_out, H_out))
                        frame_lbl[t] = cv2.resize(frame_lbl[t], (W_out, H_out))

                meta_data['scale_factor'] = sc

            ## random crop
            cropper = RandomCrop(frame_ip[0].shape[:2], (self.crop_size, self.crop_size))


            for t in range(self.sample_frames):
                frame_ip[t] = cropper(frame_ip[t])
                if self.get_lbls:
                    frame_lbl[t] = cropper(frame_lbl[t])
                    frame_syn[t] = cropper(frame_syn[t])
                if frame_ip[t].shape[1] != self.crop_size and frame_ip[t].shape[0] != self.crop_size:
                    print("[MultiFrameDataset]: size mismatch occured... =>", frame_ip[t].shape)
                if self.get_lbls:
                        if frame_lbl[t].shape[1] != self.crop_size and frame_lbl[t].shape[0] != self.crop_size:
                            print("[MultiFrameDataset]: size mismatch occured... =>", frame_lbl[t].shape)
                        if frame_syn[t].shape[1] != self.crop_size and frame_syn[t].shape[0] != self.crop_size:
                            print("[MultiFrameDataset]: size mismatch occured... =>", frame_syn[t].shape)
            meta_data['crop_coords'] = crop_thingies


            if self.geometry_aug:
                #meta_data['rotation'] = False
                ### random rotate
                rotate = random.randint(0, 3)
                if rotate != 0:
                    for t in range(self.sample_frames):
                        frame_ip[t] = np.rot90(frame_ip[t], rotate)
                        if self.get_lbls:
                            frame_syn[t] = np.rot90(frame_syn[t], rotate)
                            frame_lbl[t] = np.rot90(frame_lbl[t], rotate)
                    #meta_data['rotation'] = True

                ## horizontal flip
                if np.random.random() >= 0.5:
                    for t in range(self.sample_frames):
                        frame_ip[t] = cv2.flip(frame_ip[t], flipCode=0)
                        if self.get_lbls:
                            frame_syn[t] = cv2.flip(frame_syn[t], flipCode=0)
                            frame_lbl[t] = cv2.flip(frame_lbl[t], flipCode=0)
                    #meta_data['hflip'] = True


            if self.order_aug:
                ## reverse temporal order
                #meta_data['order'] = "normal"
                if np.random.random() >= 0.5:
                    #meta_data['order'] = "reversed"
                    frame_ip.reverse()
                    if self.get_lbls:
                        frame_syn.reverse()
                        frame_lbl.reverse()

        elif self.mode == "val":
            input_dir = self.dataset_task_list[index]
            #lbl_dir = self.dataset_task_list[index][1]
            meta_data['unstable_video_path'] = input_dir
            meta_data['video_name'] = input_dir
            #meta_data['stable_video_path'] = lbl_dir

            ## sample from T to T + #sample_frames - 1
            frame_ip = []
            #frame_lbl = []
            ip_frame_list = natsort.natsorted(glob.glob(os.path.join(input_dir, "*.*")))
            #lbl_frame_list = natsort.natsorted(glob.glob(os.path.join(lbl_dir, "*.*")))
            for t in range(0, self.sample_frames):
                frame_ip.append(read_img(ip_frame_list[t]))
                #frame_lbl.append(read_img(lbl_frame_list[t]))
            meta_data['ip_frame_paths'] = ip_frame_list

        ### convert (H, W, C) array to (C, H, W) tensor
        X = []
        if self.mode == "train":
            if self.get_lbls:
                X2 = []
                Y = []
            for t in range(len(frame_ip)):
                X.append(torch.from_numpy(frame_ip[t].transpose(2, 0, 1).astype(np.float32)))
                if self.get_lbls:
                    X2.append(torch.from_numpy(frame_syn[t].transpose(2, 0, 1).astype(np.float32)))
                    Y.append(torch.from_numpy(frame_lbl[t].transpose(2, 0, 1).astype(np.float32)))


            if self.get_lbls:
                return {'X': X, 'X2': X2, 'Y': Y, 'meta_data': meta_data}
            else:
                return {'X': X, 'meta_data': meta_data}

        else:
            for t in range(len(frame_ip)):
                X.append(torch.unsqueeze(torch.from_numpy(frame_ip[t].transpose(2, 0, 1).astype(np.float32)), 0))

            return {'X': X, 'meta_data': meta_data}

class SubsetSequentialSampler(Sampler):

    def __init__(self, indices):
        self.indices = indices

    def __iter__(self):
        return (self.indices[i] for i in range(len(self.indices)))

    def __len__(self):
        return len(self.indices)

def create_data_loader(data_set, mode, train_epoch_size= 1000, batch_size= 4, threads= 8):

    ### generate random index
    if mode == 'train':
        total_samples = train_epoch_size * batch_size
    else:
        raise ValueError("Only train mode is implemented at the moment...")

    num_epochs = int(math.ceil(float(total_samples) / len(data_set)))

    indices = np.random.permutation(len(data_set))
    indices = np.tile(indices, num_epochs)
    indices = indices[:total_samples]

    ### generate data sampler and loader
    sampler = SubsetSequentialSampler(indices)
    if mode == "train":
        data_loader = DataLoader(dataset= data_set, num_workers= threads, batch_size= batch_size, sampler= sampler, pin_memory= True)

    return data_loader

###LOSS

In [ ]:
# import os
# import numpy as np
# import cv2
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import contextual_loss as cx
# from model_cs import CoarseStabilizerInferReady


# def gradient_loss(gen_frames, gt_frames, alpha=1):

#     def gradient(x):
#         # idea from tf.image.image_gradients(image)
#         # https://github.com/tensorflow/tensorflow/blob/r2.1/tensorflow/python/ops/image_ops_impl.py#L3441-L3512
#         # x: (b,c,h,w), float32 or float64
#         # dx, dy: (b,c,h,w)

#         h_x = x.size()[-2]
#         w_x = x.size()[-1]
#         # gradient step=1
#         left = x
#         right = F.pad(x, [0, 1, 0, 0])[:, :, :, 1:]
#         top = x
#         bottom = F.pad(x, [0, 0, 0, 1])[:, :, 1:, :]

#         # dx, dy = torch.abs(right - left), torch.abs(bottom - top)
#         dx, dy = right - left, bottom - top
#         # dx will always have zeros in the last column, right-left
#         # dy will always have zeros in the last row,    bottom-top
#         dx[:, :, :, -1] = 0
#         dy[:, :, -1, :] = 0

#         return dx, dy

#     # gradient
#     gen_dx, gen_dy = gradient(gen_frames)
#     gt_dx, gt_dy = gradient(gt_frames)
#     #
#     grad_diff_x = torch.abs(gt_dx - gen_dx)
#     grad_diff_y = torch.abs(gt_dy - gen_dy)

#     # condense into one tensor and avg
#     return torch.mean(grad_diff_x ** alpha + grad_diff_y ** alpha)


# def gram(x):
# 	b,c,h,w = x.size();
# 	x = x.view(b*c, -1);
# 	return torch.mm(x, x.t())


# class outer_loop_loss_affine(nn.Module):
# 	def __init__(self, flownet, interpolator, summary_writer= None):
# 		super(outer_loop_loss_affine, self).__init__()
# 		self.crit = cx.ContextualLoss(use_vgg= True, vgg_layer='relu3_4', band_width= 0.1, loss_type= 'cosine').cuda()
# 		self.flownet = flownet
# 		self.summary_writer = summary_writer
# 		self.writer_counter = 0
# 		self.interpolator = interpolator
# 		self.l2 = nn.MSELoss(size_average=True)
# 		self.l1 = nn.L1Loss()
# 		#self.affine = CoarseStabilizerInferReady(self.flownet)


# 	def forward(self, semi_stable, stable):
# 		#### Flow loss: 	loss between semi stable and transformed (Working: minimize the motion (flow) between the generated and transformed (coarsely stable) frames)...
# 		#### Pixel loss:	pixel space loss between the generated and transformed stable...

# 		motion_between_gen_and_transformed = 0
# 		pixel_loss = 0


# 		for i in range(1, len(semi_stable)):
# 			#### Flow loss
# 			flow_stable = self.flownet.estimateFlowFull(stable[i - 1], stable[i])
# 			flow_unstable = self.flownet.estimateFlowFull(semi_stable[i - 1], semi_stable[i])
# 			motion_between_gen_and_transformed = motion_between_gen_and_transformed + self.l2(flow_unstable, flow_stable)


# 			#motion_between_gen_and_transformed = motion_between_gen_and_transformed + self.l2(torch.zeros_like(flow).cuda(), flow)

# 			#### Pixel loss
# 			pixel_loss += self.crit(semi_stable[i], stable[i])

# 		curr_loss = (10.0*torch.abs(motion_between_gen_and_transformed) + 100.0*pixel_loss)/(len(semi_stable) )



# 		if self.summary_writer is not None:
# 			self.summary_writer.add_scalar('outer_loop_loss/pixel_loss', pixel_loss.item(), self.writer_counter)
# 			self.summary_writer.add_scalar('outer_loop_loss/motion_loss', torch.abs(motion_between_gen_and_transformed).item(), self.writer_counter)
# 			self.writer_counter += 1

# 		return curr_loss

# 	def epe(self, tenFlow, tenTruth):
# 		tenEpe = torch.mean(torch.sqrt((tenFlow[:, 0:1, :, :] - tenTruth[:, 0:1, :, :])**2 + (tenFlow[:, 1:2, :, :] - tenTruth[:, 1:2, :, :])**2))
# 		return tenEpe


# class inner_loop_loss_perc_affine_another_try(nn.Module):
# 	def __init__(self, loss_fn, interpolator, flownet, summary_writer= None, use_vgg= True):
# 		super(inner_loop_loss_perc_affine_another_try, self).__init__()
# 		self.summary_writer = summary_writer
# 		self.writer_counter = 0
# 		self.crit = loss_fn
# 		self.cx = cx.ContextualLoss(use_vgg= True, vgg_layer='relu3_4', band_width= 0.1, loss_type= 'cosine').cuda()
# 		self.interpolator = interpolator
# 		self.flownet = flownet
# 		if use_vgg:
# 			self.use_vgg = True
# 			self.norm_vgg = utils.normalize_ImageNet_stats
# 			self.VGG = networks.Vgg16(requires_grad= False)
# 			self.VGG = self.VGG.cuda()
# 			VGGLayers = [int(layer) for layer in list("4")]
# 			VGGLayers.sort()
# 			self.VGGLayers = [layer-1 for layer in list(VGGLayers)]

# 		self.affine = CoarseStabilizerInferReady(self.flownet)


# 	def forward(self, semi_stable, unstable):
# 		#with torch.no_grad():
# 		unstable_txs = []
# 		unstable_affs = []
# 		for i in range(1, len(unstable)):
# 			tx, op = self.affine(unstable[0], unstable[i])
# 			unstable_txs.append(tx)
# 			unstable_affs.append(op)

# 		#### Flow loss: 	loss between semi stable and transformed (Working: minimize the motion (flow) between the generated and transformed (coarsely stable) frames)...
# 		#### Affine loss: 	loss between the original and lerped transforms...
# 		#### Pixel loss:	pixel space loss between the generated and transformed stable...
# 		motion_between_gen_and_transformed = 0
# 		aff_loss = 0
# 		pixel_loss = 0
# 		vgg_loss = 0
# 		for i in range(1, len(semi_stable)):
# 			#### Flow loss
# 			flow = self.flownet(unstable_txs[i - 1], semi_stable[i])

# 			motion_between_gen_and_transformed = motion_between_gen_and_transformed + (torch.mean(flow))

# 			#### Pixel loss + Affine loss
# 			pixel_loss = pixel_loss + self.cx(semi_stable[i], unstable[i])

# 			f_x = self.norm_vgg(semi_stable[i])
# 			f_y = self.norm_vgg(unstable_txs[i - 1])
# 			f_yy = self.norm_vgg(unstable[i])
# 			for l in self.VGGLayers:
# 				f_x = self.VGG(f_x, self.VGGLayers[-1])
# 				f_y = self.VGG(f_y, self.VGGLayers[-1])
# 				f_yy = self.VGG(f_yy, self.VGGLayers[-1])
# 				vgg_loss = vgg_loss + self.crit(f_x[l], f_y[l])
# 				vgg_loss = vgg_loss + self.crit(gram(f_x[l]), gram(f_yy[l]))
# 			#if i < len(semi_stable) - 1:
# 			#	y_, _ = self.interpolator(semi_stable[i - 1].clone().detach(), semi_stable[i + 1].clone().detach())
# 			#	pixel_loss = pixel_loss + self.crit(semi_stable[i], y_)

# 		if self.summary_writer is not None:
# 			self.summary_writer.add_scalar('inner_loop_loss/pixel_loss', pixel_loss.item(), self.writer_counter)
# 			self.summary_writer.add_scalar('inner_loop_loss/motion_loss', torch.abs(motion_between_gen_and_transformed).item(), self.writer_counter)
# 			self.summary_writer.add_scalar('inner_loop_loss/perc_loss', vgg_loss.item(), self.writer_counter)
# 			self.writer_counter += 1
# 		curr_loss = (pixel_loss) + 10*(torch.abs(motion_between_gen_and_transformed) + vgg_loss)

# 		return curr_loss

ModuleNotFoundError: No module named 'networks'

###TRAIN + EVAL

In [ ]:
# import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# os.chdir(os.path.dirname(__file__))
# print("CWD:", os.getcwd())

# import argparse
# import numpy as np
# import cv2
# from tqdm import tqdm
# import natsort
# import glob
# import re
# from collections import OrderedDict
# import time
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import higher
# import kornia as K
# from tensorboardX import SummaryWriter
# from datasets import MultiFramesDataset, create_data_loader
# from losses import inner_loop_loss_perc_affine_another_try, outer_loop_loss_affine
# import utils
# import shutil

# #### Some useless counters, cuz I'm lazy af...
# INNER_ITER_COUNTER = 0
# OUTER_ITER_COUNTER = 0
# LBLS_GIVEN = True
# USE_PREV_CROP_LOCS = False
# PREV_CROP_H = 0
# PREV_CROP_W = 0


# def get_model(opts):
#     if opts.model == "DMBVS":
#         from models.model_enet import Generator as Model
#         model = Model(in_channels=15, out_channels=3, residual_blocks=64)
#         state_dict = torch.load(os.path.join("pretrained_models", opts.ckp_name))['model']
#         ##### create new OrderedDict that does not contain `module.`
#         new_state_dict = OrderedDict()
#         for k, v in state_dict.items():
#             name = k[7:] # remove `module.`
#             new_state_dict[name] = v
#         # load params
#         model.load_state_dict(new_state_dict)
#     elif opts.model == 'difrint':
#         from models.difrint import DIFNet_ours as Model
#         model = Model()
#         state_dict = torch.load('./pretrained_models/DIFNet2.pth')
#         ##### create new OrderedDict that does not contain `module.`
#         new_state_dict = OrderedDict()
#         for k, v in state_dict.items():
#             name = k[7:] # remove `module.`
#             new_state_dict[name] = v
#         # load params
#         model.load_state_dict(new_state_dict)
#         for i, child in enumerate(model.children()):
#             if i < 2:
#                 for param in child.parameters():
#                     param.requires_grad = False
#                 print(f'Freezed {child.__class__}')
#             else:
#                 pass
#     else:
#         raise Exception("Model not implemented: (%s)" %opts.model)
#     print('Loaded model state dict')
#     return model


# def get_data_loaders(opts):
#     bc_dir = opts.data_dir
#     opts_train = opts
#     opts_train.data_dir = opts.train_data_dir
#     train_dataset = MultiFramesDataset(mode= 'train', opts= opts_train, get_lbls= LBLS_GIVEN, prev_iter_counter= OUTER_ITER_COUNTER)
#     opts.data_dir = bc_dir
#     test_dataset = MultiFramesDataset(mode= "val", opts= opts)
#     train_data_loader = create_data_loader(train_dataset, mode= "train",
#         batch_size= opts.batch_size, threads= opts.threads, train_epoch_size= opts.train_epoch_size)
#     test_data_loader = utils.create_data_loader(test_dataset, opts= opts, mode= "val")

#     return train_data_loader, test_dataset # intentionally returning dataset instead of dataloader to gauge the performance on a single video [0]


# def ip_gen_dif(frames, frames_op, start_id, device= "cuda", return_orig= False):
#     if len(frames_op) == 0:
#         frames_op.append(frames[start_id].clone())
#     if device == "cuda":
#         ip = torch.cat([frames_op[-1].cpu(),
#             frames[start_id + 1],
#             frames[start_id + 2]], 1).cuda()
#         if return_orig:
#             temp = frames[start_id + 1].clone().cuda()
#     else:
#         ip = torch.cat([frames_op[-1].cpu(),
#             frames[start_id + 1],
#             frames[start_id + 2]], 1)
#         if return_orig:
#             temp = frames[start_id + 1].clone()
#     if return_orig:
#         return ip, temp
#     else:
#         return ip


# def ip_gen_dif_test(frames, frames_op, start_id, device= "cuda", patch_size= 320): #### For test time adaptation, does 5 adaptation steps on randomly cropped data and then we use the above one for full frame retrieval...
#     b, c, h, w = frames[0].shape
#     global USE_PREV_CROP_LOCS
#     global PREV_CROP_H
#     global PREV_CROP_W

#     if not USE_PREV_CROP_LOCS:
#         if h == patch_size:
#             h_ = 0
#         else:
#             h_ = np.random.randint(0, h - patch_size)
#         w_ = np.random.randint(0, w - patch_size)
#         PREV_CROP_H = h_
#         PREV_CROP_W = w_
#         USE_PREV_CROP_LOCS = True
#     else:
#         h_ = PREV_CROP_H
#         w_ = PREV_CROP_H

#     if len(frames_op) == 0:
#         frames_op.append(utils.tensor_crop(frames[start_id], h_, w_, patch_size))
#     if device == "cuda":
#         ip = torch.cat([frames_op[-1].cpu(),
#             utils.tensor_crop(frames[start_id + 1], h_, w_, patch_size),
#             utils.tensor_crop(frames[start_id + 2], h_, w_, patch_size)], 1).cuda()
#         spt = utils.tensor_crop(frames[start_id + 1], h_, w_, patch_size).cuda()
#     else:
#         ip = torch.cat([frames_op[-1].cpu(),
#             utils.tensor_crop(frames[start_id + 1], h_, w_, patch_size),
#             utils.tensor_crop(frames[start_id + 2], h_, w_, patch_size)], 1)
#         spt = utils.tensor_crop(frames[start_id + 1], h_, w_, patch_size)

#     return ip, spt


# def ip_gen_dif_for_outer(frames, frames_op, frames_lbl, start_id, device= "cuda", return_orig= True, resize= True):
#     if len(frames_op) == 0:
#         frames_op.append(frames[start_id])
#     if device == "cuda":
#         ip = torch.cat([frames_op[-1].cpu(),
#             frames[start_id + 1],
#             frames[start_id + 2]], 1).cuda()
#         if return_orig:
#             temp = frames_lbl[start_id + 1].clone().cuda()
#     else:
#         ip = torch.cat([frames_op[-1].cpu(),
#             frames[start_id + 1],
#             frames[start_id + 2]], 1)
#         if return_orig:
#             temp = frames_lbl[start_id + 1].clone()
#     if return_orig:
#         if resize:
#             ip = nn.functional.interpolate(ip, (192, 192))
#             temp = nn.functional.interpolate(temp, (192, 192))
#         return ip, temp
#     else:
#         ip = nn.functional.interpolate(ip, (192, 192))
#         return ip


# def test_no_adaptation(db, net, device, L_in, epoch, opts):
#     if not os.path.exists("./output_" + opts.model_name + "/" + str(epoch) + "/"):
#         os.makedirs("./output_" + opts.model_name + "/" + str(epoch) + "/")

#     if not os.path.exists("./output_" + opts.model_name + "/target/"):
#         os.makedirs("./output_" + opts.model_name + "/target/")

#     net.train()
#     net.pwc.eval()

#     final_op_list = []
#     final_ip_list = []
#     frame_pointer = 0
#     interim_ops = []
#     test_batch = db[0]
#     frames = test_batch['X']

#     pbar = tqdm(total= len(frames) - 5 - 5)
#     while(frame_pointer < len(frames) - 5 - 5):
#         with torch.no_grad():
#             net.eval()
#             ip_ = ip_gen_dif(frames, interim_ops, frame_pointer, return_orig= False)
#             op_ = torch.clamp(net(ip_), min= 0.0, max= 1.0)
#             interim_ops.append(op_.clone())
#             final_op_list.append((utils.tensor2img(op_)*255).astype(np.uint8))
#             final_ip_list.append((utils.tensor2img(frames[frame_pointer + 1])*255).astype(np.uint8))
#             net.train()
#         pbar.update(1)
#         frame_pointer += 1
#     pbar.close()
#     for i in range(len(final_op_list)):
#         cv2.imwrite("./output_" + opts.model_name + "/" + str(epoch) + "/" + str(i) + ".png", final_op_list[i])
#     for i in range(len(final_ip_list)):
#         cv2.imwrite("./output_" + opts.model_name + "/target/" + str(i) + ".png", final_ip_list[i])

#     return final_op_list


# def eval_no_adaptation(db, net, device, L_in, epoch, opts):
#     if not os.path.exists("./output_" + opts.model_name + "/" + str(epoch) + "_no_adapt/"):
#         os.makedirs("./output_" + opts.model_name + "/" + str(epoch) + "_no_adapt/")

#     if not os.path.exists("./output_" + opts.model_name + "/target/"):
#         os.makedirs("./output_" + opts.model_name + "/target/")

#     net.train()
#     net.pwc.eval()

#     final_op_list = []
#     final_ip_list = []
#     frame_pointer = 0
#     interim_ops = []
#     test_batch = db[0]
#     frames = test_batch['X']

#     pbar = tqdm(total= len(frames) - 5 - 5)
#     while(frame_pointer < len(frames) - 5 - 5):
#         with torch.no_grad():
#             net.eval()
#             ip_ = ip_gen_dif(frames, interim_ops, frame_pointer, return_orig= False)
#             op_ = torch.clamp(net(ip_), min= 0.0, max= 1.0)
#             interim_ops.append(op_.clone())
#             final_op_list.append((utils.tensor2img(op_)*255).astype(np.uint8))
#             final_ip_list.append((utils.tensor2img(frames[frame_pointer + 1])*255).astype(np.uint8))
#             net.train()
#         pbar.update(1)
#         frame_pointer += 1
#     pbar.close()
#     for i in range(len(final_op_list)):
#         cv2.imwrite("./output_" + opts.model_name + "/" + str(epoch) + "_no_adapt/" + str(i) + ".png", final_op_list[i])
#     for i in range(len(final_ip_list)):
#         cv2.imwrite("./output_" + opts.model_name + "/target/" + str(i) + ".png", final_ip_list[i])

#     return final_op_list


# def test(db, net, device, L_in, epoch, opts):
#     if not os.path.exists("./output_" + opts.model_name + "/" + str(epoch) + "/"):
#         os.makedirs("./output_" + opts.model_name + "/" + str(epoch) + "/")
#     net.train()
#     net.pwc.eval()
#     #n_test_iter = 20 ### generate 20 frames for test only

#     final_op_list = []
#     final_ip_list = []
#     frame_pointer = 0

#     test_batch = db[0]
#     frames = test_batch['X']

#     lr = 5e-7
#     adaptation_number = 3
#     optimizer = optim.Adam(net.parameters(), lr)
#     pbar = tqdm(total= len(frames) - 5 - 5)
#     global USE_PREV_CROP_LOCS
#     while(frame_pointer < len(frames) - 5 - 5):
#         interim_op = []
#         USE_PREV_CROP_LOCS = False
#         for _ in range(adaptation_number):
#             optimizer.zero_grad()
#             ip1, spt_ip1 = ip_gen_dif_test(frames, interim_op, start_id= frame_pointer, patch_size= opts.crop_size_adap)
#             spt_op1 = torch.clamp(net(ip1), min= 0.0, max= 1.0)
#             interim_op.append(spt_op1.clone())

#             ip2, spt_ip2 = ip_gen_dif_test(frames, interim_op, start_id= frame_pointer + 1, patch_size= opts.crop_size_adap)
#             spt_op2 = torch.clamp(net(ip2), min= 0.0, max= 1.0)
#             interim_op.append(spt_op2.clone())

#             ip3, spt_ip3 = ip_gen_dif_test(frames, interim_op, start_id= frame_pointer + 2, patch_size= opts.crop_size_adap)
#             spt_op3 = torch.clamp(net(ip3), min= 0.0, max= 1.0)
#             interim_op.append(spt_op3.clone())

#             ip4, spt_ip4 = ip_gen_dif_test(frames, interim_op, start_id= frame_pointer + 3, patch_size= opts.crop_size_adap)
#             spt_op4 = torch.clamp(net(ip4), min= 0.0, max= 1.0)
#             interim_op.append(spt_op4.clone())

#             ip5, spt_ip5 = ip_gen_dif_test(frames, interim_op, start_id= frame_pointer + 4, patch_size= opts.crop_size_adap)
#             spt_op5 = torch.clamp(net(ip5), min= 0.0, max= 1.0)
#             interim_op.append(spt_op5.clone())
#             #spt_loss = L_in(spt_op1, spt_op2, spt_op3, spt_ip1, spt_ip2, spt_ip3)
#             spt_loss = 0.5 * L_in([spt_op1, spt_op2, spt_op3, spt_op4, spt_op5], [spt_ip1, spt_ip2, spt_ip3, spt_ip4, spt_ip5])

#             spt_loss.backward(retain_graph= True)
#             optimizer.step()


#         pbar.update(1)
#         frame_pointer += 1
#     pbar.close()
#     frame_pointer = 0
#     interim_ops = []
#     while(frame_pointer < len(frames) - 5 - 5):
#         with torch.no_grad():
#             net.eval()
#             ip_ = ip_gen_dif(frames, interim_ops, frame_pointer)
#             op_ = torch.clamp(net(ip_), min= 0.0, max= 1.0)
#             interim_ops.append(op_.clone())
#             #frames[frame_pointer] = op_ #### frame recurrence...
#             final_op_list.append((utils.tensor2img(op_)*255).astype(np.uint8))
#             final_ip_list.append((utils.tensor2img(frames[frame_pointer + 1])*255).astype(np.uint8))
#             frame_pointer += 1
#         net.train()
#     for i in range(len(final_op_list)):
#         cv2.imwrite("./output_" + opts.model_name + "/" + str(epoch) + "/" + str(i) + ".png", final_op_list[i])

#     return final_op_list


# def train(data_loader, net, device, meta_opt, L_in, L_out, epoch, opts):
#     net.train()
#     net.pwc.eval() #### DIF ckpt contains PWCNet

#     global INNER_ITER_COUNTER
#     global OUTER_ITER_COUNTER

#     if epoch > opts.lr_offset:
#         current_lr = utils.learning_rate_decay_simple(opts, epoch)
#     else:
#         current_lr = opts.lr_init
#     for param_group in meta_opt.param_groups:
#         param_group['lr'] = current_lr

#     loss_writer.add_scalar("hyperparameters/LR", current_lr, epoch)

#     for batch_idx, batch in enumerate(data_loader, 1):
#         start_time = time.time()
#         frames = batch['X']
#         if LBLS_GIVEN:
#             frames_lbl = batch['Y']

#         frame_inds = [0, 3, 6, 9, 12] #### for 5 (with 3 disjoint samples) inner loop iterations
#         n_inner_iter = 1 #len(frame_inds) #for doing 1 inner loop optimization...




#         inner_opt = torch.optim.Adam(net.parameters(), lr=current_lr)


#         meta_opt.zero_grad()
#         for i in range(1):
#             with higher.innerloop_ctx(
#                 net, inner_opt, copy_initial_weights=False
#             ) as (fnet, diffopt):
#                 for ind in range(n_inner_iter):
#                     interim_op = []
#                     ip1, unstab1 = ip_gen_dif(frames, interim_op, start_id= frame_inds[ind], return_orig= True)
#                     spt_op1 = torch.clamp(fnet(ip1), min= 0.0, max= 1.0)
#                     interim_op.append(spt_op1.clone())

#                     ip2, unstab2 = ip_gen_dif(frames, interim_op, start_id= frame_inds[ind] + 1, return_orig= True)
#                     spt_op2 = torch.clamp(fnet(ip2), min= 0.0, max= 1.0)
#                     interim_op.append(spt_op2.clone())

#                     ip3, unstab3 = ip_gen_dif(frames, interim_op, start_id= frame_inds[ind] + 2, return_orig= True)
#                     spt_op3 = torch.clamp(fnet(ip3), min= 0.0, max= 1.0)
#                     interim_op.append(spt_op3.clone())

#                     ip4, unstab4 = ip_gen_dif(frames, interim_op, start_id= frame_inds[ind] + 3, return_orig= True)
#                     spt_op4 = torch.clamp(fnet(ip4), min= 0.0, max= 1.0)
#                     interim_op.append(spt_op4.clone())

#                     ip5, unstab5 = ip_gen_dif(frames, interim_op, start_id= frame_inds[ind] + 4, return_orig= True)
#                     spt_op5 = torch.clamp(fnet(ip5), min= 0.0, max= 1.0)
#                     interim_op.append(spt_op5.clone())


#                     spt_loss = L_in([spt_op1, spt_op2, spt_op3, spt_op4, spt_op5], [unstab1, unstab2, unstab3, unstab4, unstab5])
#                     diffopt.step(spt_loss)
#                     loss_writer.add_scalar("comb_losses/InnerLoopLoss", spt_loss.item(), INNER_ITER_COUNTER)
#                     INNER_ITER_COUNTER += 1
#                     print("Epoch:", epoch, "Outer_iter:", batch_idx, "Inner_iter:", ind, "L_in:", spt_loss.item(), "({}/{})".format(batch_idx, len(data_loader)))

#                 op_qry = []
#                 stab_qry = []

#                 interim_ops = []
#                 for q in range(6):
#                     id_for_qry = frame_inds[-1] + 3 + q
#                     if LBLS_GIVEN:
#                         ip_qry, stab = ip_gen_dif_for_outer(frames, interim_ops, frames_lbl, start_id= id_for_qry)
#                     else:
#                         ip_qry, stab = ip_gen_dif_for_outer(frames, interim_ops, frames, start_id= id_for_qry)
#                     op_qry.append(torch.clamp(fnet(ip_qry), min= 0.0, max= 1.0))
#                     interim_ops.append(op_qry[-1].clone())
#                     stab_qry.append(stab)

#                 qry_loss = L_out(op_qry, stab_qry)

#                 loss_writer.add_scalar("comb_losses/OuterLoopLoss", qry_loss.item(), OUTER_ITER_COUNTER)
#                 OUTER_ITER_COUNTER += 1
#                 qry_loss.backward()
#                 print("Epoch:", epoch, "Outer_iter:", batch_idx, "Inner_iter:", ind, "L_out:", qry_loss.item(), "({}/{})".format(batch_idx, len(data_loader)))

#             meta_opt.step()
#             torch.cuda.empty_cache()
#     torch.cuda.empty_cache()

#     return net, meta_opt

In [ ]:
# parser = argparse.ArgumentParser(description="Meta Stabilization")

# ### model options
# parser.add_argument('-model',           type=str,     default="difrint",                        help='Model to use')
# parser.add_argument('-ckp_name',        type=str,     default="best_stab_vf_v1.h5",             help='checkpoint_name')
# parser.add_argument('-model_name',      type=str,     default='dif_after_1_4',                  help='path to save model')

# ### dataset options
# parser.add_argument('-train_data_dir',  type=str,     default='../DeepStab/unstable_frames/',   help='path to train data folder')
# parser.add_argument('-train_lbl_dir',  type=str,     default='../DeepStab/stable_frames/',      help='path to train data folder')
# parser.add_argument('-data_dir',        type=str,     default='../DeepStab/unstable_frames/',   help='path to test data folder')#### change to the folder containing NUS Dataset videos
# parser.add_argument('-checkpoint_dir',  type=str,     default='checkpoints',                    help='path to checkpoint folder')
# parser.add_argument('-crop_size',       type=int,     default=192,                              help='patch size')
# parser.add_argument('-crop_size_adap',  type=int,     default=320,                              help='adaptation patch size')
# parser.add_argument('-geometry_aug',    type=int,     default=0,                                help='geometry augmentation (rotation, scaling, flipping)')
# parser.add_argument('-order_aug',       type=int,     default=1,                                help='temporal ordering augmentation')
# parser.add_argument('-scale_min',       type=float,   default=0.5,                              help='min scaling factor')
# parser.add_argument('-scale_max',       type=float,   default=2.0,                              help='max scaling factor')
# parser.add_argument('-sample_frames',   type=int,     default=63,                               help='#frames for training') #### Min number of frames in one of the deep stab scenes are 67...

# ### training options
# parser.add_argument('-solver',          type=str,     default="ADAM",                           choices=["SGD", "ADAM"],   help="optimizer")
# parser.add_argument('-momentum',        type=float,   default=0.9,                              help='momentum for SGD')
# parser.add_argument('-beta1',           type=float,   default=0.9,                              help='beta1 for ADAM')
# parser.add_argument('-beta2',           type=float,   default=0.999,                            help='beta2 for ADAM')
# parser.add_argument('-weight_decay',    type=float,   default=0,                                help='weight decay')
# parser.add_argument('-batch_size',      type=int,     default=4,                                help='training batch size')
# parser.add_argument('-train_epoch_size',type=int,     default=200,                              help='train epoch size')
# parser.add_argument('-valid_epoch_size',type=int,     default=100,                              help='valid epoch size')
# parser.add_argument('-epoch_max',       type=int,     default=1000,                             help='max #epochs')


# ### learning rate options
# parser.add_argument('-lr_init',         type=float,   default=1e-5,                             help='initial learning Rate')
# parser.add_argument('-lr_offset',       type=int,     default=10,                               help='epoch to start learning rate drop [-1 = no drop]')
# parser.add_argument('-lr_step',         type=int,     default=1,                                help='step size (epoch) to drop learning rate')
# parser.add_argument('-lr_drop',         type=float,   default=0.99,                             help='learning rate drop ratio')
# parser.add_argument('-lr_min_m',        type=float,   default=0.001,                            help='minimal learning Rate multiplier (lr >= lr_init * lr_min)')


# ### other options
# parser.add_argument('-loss',            type=str,     default="L1",                             help="Loss [Options: L1, L2]")
# parser.add_argument('-seed',            type=int,     default=9487,                             help='random seed to use')
# parser.add_argument('-threads',         type=int,     default=8,                                help='number of threads for data loader to use')
# parser.add_argument('-suffix',          type=str,     default='',                               help='name suffix')
# parser.add_argument('-gpu',             type=int,     default=1,                                help='gpu device id')
# parser.add_argument('-cpu',             action='store_true',                                    help='use cpu?')

# opts = parser.parse_args()


# #### adjust the training data folder (only if you didn't set it earlier....)
# opts.train_data_dir = "../DeepStab/unstable_frames/"


# opts.cuda = (opts.cpu != True)
# opts.lr_min = opts.lr_init * opts.lr_min_m

# ### default model name
# if opts.model_name == 'none':
#     opts.model_name = "%s_%s" %(opts.model, opts.ckp_name)

# if opts.suffix != "":
#     opts.model_name += "_%s" %opts.suffix


# opts.size_multiplier = 2 ** 6 ## Inputs to FlowNet need to be divided by 64

# print(opts)

# ### model saving directory
# opts.model_dir = os.path.join(opts.checkpoint_dir, opts.model_name)
# print("========================================================")
# print("===> Save model to %s" %opts.model_dir)
# print("========================================================")
# if not os.path.isdir(opts.model_dir):
#     os.makedirs(opts.model_dir)

# ### initialize model
# print('===> Initializing model from %s...' %opts.model)
# model = get_model(opts)

# ### Meta optimizer...
# ### initialize optimizer
# if opts.solver == 'SGD':
#     meta_opt = optim.SGD(model.parameters(), lr=opts.lr_init, momentum=opts.momentum, weight_decay=opts.weight_decay)
# elif opts.solver == 'ADAM':
#     meta_opt = optim.Adam(model.parameters(), lr=opts.lr_init, weight_decay=opts.weight_decay, betas=(opts.beta1, opts.beta2))
# else:
#     raise Exception("Not supported solver (%s)" %opts.solver)

# if opts.loss == 'L2':
#     criterion = nn.MSELoss(size_average=True)
# elif opts.loss == 'L1':
#     criterion = nn.L1Loss(size_average=True)
# else:
#     raise Exception("Unsupported criterion %s" %opts.loss)

# num_params = utils.count_network_parameters(model)


# ### resume latest model
# name_list = glob.glob(os.path.join(opts.model_dir, "model_epoch_*.pth"))
# epoch_st = 0

# if len(name_list) > 0:
#     epoch_list = []
#     for name in name_list:
#         s = re.findall(r'\d+', os.path.basename(name))[0]
#         epoch_list.append(int(s))

#     epoch_list.sort()
#     epoch_st = epoch_list[-1]


# if epoch_st > 0:

#     print('=====================================================================')
#     print('===> Resuming model from epoch %d' %epoch_st)
#     print('=====================================================================')

#     ### resume latest model and solver
#     model, meta_opt = utils.load_model(model, meta_opt, opts, epoch_st)

# else:
#     ### save epoch 0
#     utils.save_model(model, meta_opt, opts, 0)


# print('\n=====================================================================')
# print("===> Model has %d parameters" %num_params)
# print('=====================================================================')


# ### initialize loss writer
# loss_dir = os.path.join(opts.model_dir, 'loss')
# loss_writer = SummaryWriter(loss_dir)


# #Will pass it to the loss definition

# ### Load pretrained GFlowNet
# GFlowNet = GPWC().eval()
# for param in GFlowNet.parameters():
#     param.requires_grad = False

# ### convert to GPU
# device = torch.device("cuda" if opts.cuda else "cpu")
# model = model.to(device)
# GFlowNet = GFlowNet.to(device)

# ### Define losses here and pass flownet and interpolator to losses
# L_in = inner_loop_loss_perc_affine_another_try(criterion, None, GFlowNet, summary_writer= loss_writer)
# L_out = outer_loop_loss_affine(GFlowNet, None, summary_writer= loss_writer)

# train_data_loader, test_data_loader = get_data_loaders(opts)

# #print("Testing no adaptation...")
# #_ = test_no_adaptation(test_data_loader, model, device, L_in, 0, opts)
# torch.cuda.empty_cache()
# for epoch in range(1, opts.epoch_max):
#     train_data_loader, test_data_loader = get_data_loaders(opts)
#     model, meta_opt = train(train_data_loader, model, device, meta_opt, L_in, L_out, epoch, opts)
#     torch.cuda.empty_cache()
#     print("Evaluating without adaptation...")
#     _ = eval_no_adaptation(test_data_loader, model, device, L_in, epoch, opts)
#     torch.cuda.empty_cache()

#     #### Making a copy of the model weights to do adaptation for performance check, can also use deepcopy...
#     model_for_adaptation = get_model(opts).to(device)
#     model_for_adaptation.load_state_dict(model.state_dict())
#     print("Testing with adaptation...")
#     _ = test(test_data_loader, model_for_adaptation, device, L_in, epoch, opts)
#     del model_for_adaptation
#     utils.save_model(model, meta_opt, opts, epoch)
#     torch.cuda.empty_cache()

###TEST

In [ ]:
import cv2

def scale_video(input_path, output_path, target_size=(256, 256)):
    cap = cv2.VideoCapture(input_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or 'XVID' depending on your system
    out = cv2.VideoWriter(output_path, fourcc, 30.0, target_size)  # 30.0 is frames per second

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        resized_frame = cv2.resize(frame, target_size)
        out.write(resized_frame)

    cap.release()
    out.release()

In [ ]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim

def calculate_stability_score(original_video_path, stabilized_video_path):
    cap_orig = cv2.VideoCapture(original_video_path)
    cap_stab = cv2.VideoCapture(stabilized_video_path)

    stability_scores = []

    while True:
        ret_orig, frame_orig = cap_orig.read()
        ret_stab, frame_stab = cap_stab.read()

        if not ret_orig or not ret_stab:
            break

        frame_orig_gray = cv2.cvtColor(frame_orig, cv2.COLOR_BGR2GRAY)
        frame_stab_gray = cv2.cvtColor(frame_stab, cv2.COLOR_BGR2GRAY)

        score, _ = ssim(frame_orig_gray, frame_stab_gray, full=True)
        stability_scores.append(score)

    cap_orig.release()
    cap_stab.release()

    return np.mean(stability_scores)


####So sánh video đầu ra với stable video dataset

In [ ]:
input_video_path = '/content/DeepStab/stable/1.avi'
output_video_path = '/content/scaled_video.avi'
scale_video(input_video_path, output_video_path)

In [ ]:
original_video = '/content/scaled_video.avi'
stabilized_video = '/content/stable_result.avi'
stability_score = calculate_stability_score(original_video, stabilized_video)
print(f'Stability Score: {stability_score}')

Stability Score: 0.3257793336397868


####So sánh video đầu ra với unstable video dataset

In [ ]:
input_video_path = '/content/DeepStab/unstable/1.avi'
output_video_path = '/content/unstable_scaled_video.avi'
scale_video(input_video_path, output_video_path)

In [ ]:
original_video = '/content/unstable_scaled_video.avi'
stabilized_video = '/content/stable_result.avi'
stability_score = calculate_stability_score(original_video, stabilized_video)
print(f'Stability Score: {stability_score}')

Stability Score: 0.6711860836525776


###COnvert rt 

In [ ]:
import torch
import torch.quantization
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit

# Assuming `model` is your PyTorch model
model = CAIN(training=False, depth=3)  # Example model
model.eval()

# Apply dynamic quantization
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

# Save the quantized model
torch.save(quantized_model.state_dict(), "quantized_model.pth")
#Convert the Quantized Model to ONNX
# Đầu vào giả cho mô hình CAIN
dummy_input_cain1 = torch.randn(1, 3, 256, 256).cuda()  # Điều chỉnh kích thước nếu cần
dummy_input_cain2 = torch.randn(1, 3, 256, 256).cuda()  # Đầu vào thứ hai cho CAIN

# Xuất mô hình CAIN sang ONNX
torch.onnx.export(
    cain,
    (dummy_input_cain1, dummy_input_cain2),  # Đầu vào giả
    "cain_model.onnx",  # Tên tệp xuất ra
    opset_version=11,
    input_names=['input1', 'input2'],
    output_names=['output']
)
print('Mô hình CAIN đã được xuất sang định dạng ONNX.')

# Đầu vào giả cho mô hình ResNet
dummy_input_resnet = torch.randn(1, 15, 256, 256).cuda()  # Điều chỉnh kích thước nếu cần

# Xuất mô hình ResNet sang ONNX
torch.onnx.export(
    resnet,
    dummy_input_resnet,  # Đầu vào giả
    "resnet_model.onnx",  # Tên tệp xuất ra
    opset_version=11,
    input_names=['input'],
    output_names=['output']
)
print('Mô hình ResNet đã được xuất sang định dạng ONNX.')

#trtexec --onnx=quantized_model.onnx --saveEngine=quantized_model.trt --fp16 (down to fp32->fp16)
trtexec --onnx=quantized_model.onnx --saveEngine=quantized_model.trt --fp16


